In [1]:
from vantage6.algorithm.tools.mock_client import MockAlgorithmClient
import numpy as np
import os; os.chdir("..")
import warnings
import pandas as pd
warnings.filterwarnings("ignore")

In [2]:
data_dir = os.path.join(os.getcwd(), "./vtg_km_he", "local")
data_path = os.path.join(data_dir, "data_test.csv")
df = pd.read_csv(data_path)
df

,0,1,2,3,4,C,T
0,-0.04518,-0.15407,-0.38322,-0.16410,0.18149,1.0,1.0
1,0.13559,0.43563,0.01046,-0.06103,0.22061,0.0,25.0
2,0.25824,0.25285,0.33758,0.23398,-0.12381,0.0,53.0
3,-0.36326,-0.14392,-0.30458,0.25528,0.39562,1.0,12.0
4,-0.42186,-0.25667,0.23794,-0.27736,0.12835,1.0,31.0
...,...,...,...,...,...,...,...
195,-0.15622,0.36398,0.18487,-0.32684,-0.07903,1.0,52.0
196,0.33239,0.37361,0.30750,0.02121,-0.33076,1.0,28.0
197,0.35748,-0.20104,-0.41990,0.06762,-0.27252,1.0,23.0
198,-0.43103,0.13096,0.35723,0.44050,0.05443,1.0,6.0


In [12]:
# Partitioning data
# data_dir = os.path.join(os.getcwd(), "vtg_km_he", "local")
# data_path = os.path.join(data_dir, "data_test.csv")
# data1_path = os.path.join(data_dir, "data1.csv")
# data2_path = os.path.join(data_dir, "data2.csv")
# print(data2_path)

# df = pd.read_csv(data_path)
# df["COHORT_DEFINITION_ID"] = np.random.randint(low=0, high=2, size=(df.shape[0],))
# df = df.rename(columns={'T':'TIME_AT_RISK','C':'MORTALITY_FLAG'}, inplace=False) 
# df = df.sample(frac=1) # Shuffle data
# display(df)


# df[0:int(len(df)/2)].to_csv(data1_path)
# df[int(len(df)/2):].to_csv(data2_path)

/Users/ivanzhovannik/Desktop/packages/vtg_km_he/vtg_km_he/local/data2.csv


,0,1,2,3,4,MORTALITY_FLAG,TIME_AT_RISK,COHORT_DEFINITION_ID
32,0.05049,-0.41221,0.11489,-0.41616,-0.11797,1.0,23.0,1
118,-0.37277,-0.38252,-0.08142,0.31499,0.21423,1.0,2.0,0
121,0.22860,-0.38002,-0.17411,0.28001,0.24775,1.0,22.0,0
181,0.32895,-0.23834,-0.29645,-0.31416,-0.01254,0.0,42.0,1
71,-0.09517,-0.22056,-0.22282,-0.39612,0.06437,1.0,24.0,1
...,...,...,...,...,...,...,...,...
49,0.00473,-0.19929,-0.33422,0.07149,-0.26711,0.0,21.0,0
175,0.33874,-0.33529,-0.38023,-0.18039,-0.00673,1.0,14.0,1
74,0.43118,0.13600,0.27607,-0.03319,0.03760,0.0,24.0,0
43,0.29519,-0.37410,0.42146,-0.00445,0.39778,0.0,54.0,0


In [13]:
dataset_1 = {"database": "./vtg_km_he/local/data1.csv", "db_type": "csv"}
dataset_2 = {"database": "./vtg_km_he/local/data2.csv", "db_type": "csv"}
org_ids = ids = [0, 1]
display(dataset_2)

{'database': './vtg_km_he/local/data2.csv', 'db_type': 'csv'}

In [14]:
client = MockAlgorithmClient(
    datasets = [[dataset_1], [dataset_2]],
    organization_ids=org_ids,
    module="vtg_km_he"
)

In [15]:
organizations = client.organization.list()
org_ids = ids = [organization["id"] for organization in organizations]

In [16]:
average_task = client.task.create(
    input_={
        'master': 1,
        'method': 'master',
        'kwargs': {'time_column': 'TIME_AT_RISK','censor_column':'MORTALITY_FLAG'}
    },
    organizations=[org_ids[0]]
)

info > Collecting information on participating organizations
info > Sending task to organizations [0, 1]
info > Collecting unique event times
info > Sending task to organizations [0, 1]
info > Waiting for results
info > Mocking waiting for results
info > Results obtained for get_unique_event_times!
info > Collected unique event times for 2 organization(s)
info > Collecting local event tables
info > Sending task to organizations [0, 1]
info > Sample size 100
info > Sample size 100
info > Waiting for results
info > Mocking waiting for results
info > Results obtained for get_km_event_table!
info > Collected local event tables for 2 organization(s)
info > Aggregating event tables
info > Kaplan-Meier curve has been computed successfully


In [17]:
results = client.result.get(average_task.get("id"))

In [18]:
results['kaplanMeier'] = pd.read_json(results['kaplanMeier'])
results['local_event_tables'] = [pd.read_json(value) for value in results['local_event_tables']]
df_events = results['kaplanMeier']
try:
    results['kaplanMeier'].to_csv('results.csv')
except:
    results['kaplanMeierLR'].to_csv('results_lr.csv')
    results['kaplanMeierHR'].to_csv('results_hr.csv')